In [ ]:
# Creating Adverarial training dataset for FGSM Attack

# import os
# from shutil import copyfile

# original_data_path = 'data/TB_data'
# adversarial_data_path = 'attack_images/Test_attacks_FGSM'
# concatenated_data_path = 'adv_data/TB_adversarial_data'

# if not os.path.exists(concatenated_data_path):
#     os.makedirs(concatenated_data_path)

# classes = ['Normal', 'Tuberculosis']
# sets = ['training', 'testing', 'validation']

# for set_name in sets:
#     for class_name in classes:
#         original_folder_path = os.path.join(original_data_path, set_name, class_name)
#         concatenated_folder_path = os.path.join(concatenated_data_path, set_name, class_name)
        
#         if not os.path.exists(concatenated_folder_path):
#             os.makedirs(concatenated_folder_path)
        
#         # Copy original files
#         for file_name in os.listdir(original_folder_path):
#             src = os.path.join(original_folder_path, file_name)
#             dst = os.path.join(concatenated_folder_path, file_name)
#             copyfile(src, dst)
        
#         # Copy adversarial files to train set only
#         if set_name == 'training':
#             adversarial_folder_path = os.path.join(adversarial_data_path, class_name)
#             for file_name in os.listdir(adversarial_folder_path):
#                 src = os.path.join(adversarial_folder_path, file_name)
#                 dst = os.path.join(concatenated_folder_path, file_name)
#                 copyfile(src, dst)


In [2]:
# # Creating Adverarial training dataset for all Attacks
# import os
# from shutil import copyfile

# original_data_path = 'data/TB_data'
# attack_data_path = 'attack_images'
# concatenated_data_path = 'adv_data/TB_adversarial_data_all'

# if not os.path.exists(concatenated_data_path):
#     os.makedirs(concatenated_data_path)

# classes = ['Normal', 'Tuberculosis']
# sets = ['training', 'testing', 'validation']
# attack_folders = [folder for folder in os.listdir(attack_data_path) if folder.startswith('Test_attacks_')]

# for set_name in sets:
#     for class_name in classes:
#         original_folder_path = os.path.join(original_data_path, set_name, class_name)
#         concatenated_folder_path = os.path.join(concatenated_data_path, set_name, class_name)

#         if not os.path.exists(concatenated_folder_path):
#             os.makedirs(concatenated_folder_path)

#         # Copy original files
#         for file_name in os.listdir(original_folder_path):
#             src = os.path.join(original_folder_path, file_name)
#             dst = os.path.join(concatenated_folder_path, file_name)
#             copyfile(src, dst)

#         # Copy adversarial files to train set only
#         if set_name == 'training':
#             for attack_folder in attack_folders:
#                 adversarial_folder_path = os.path.join(attack_data_path, attack_folder, class_name)
#                 for file_name in os.listdir(adversarial_folder_path):
#                     src = os.path.join(adversarial_folder_path, file_name)
#                     dst = os.path.join(concatenated_folder_path, file_name)
#                     copyfile(src, dst)

In [4]:
# import functions
# import predictions
# from predictions import Prediction
# from features import Features
import mlp
from mlp import Classifier
from data import dataset as data
# import attacks
import torch
import pandas as pd
import numpy as np
import torch.nn as nn 
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns 
import random
import pickle
import torch 
from sklearn.metrics import auc
from autoattack import AutoAttack
from torchvision.utils import save_image
import math
from matplotlib.backends.backend_pdf import PdfPages
# import train_model
import os
# from ensemble_whitebox import New
import timm 
import torchsummary
import sys
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from torchsummary import summary
import pickle

def countParams(model):
    total_params = sum(param.numel() for param in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Total Params: ", total_params)
    print("Trainable Params: ", trainable_params)

In [5]:
root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= 30)

['number of train images is 10500', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']


# 1. Training mlp modules

In [3]:
from mlp import Classifier

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = Classifier(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs): 
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/mlps/mlp_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 6360', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 212/212 [00:30<00:00,  6.93it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.75
train_loss = 5.23
Epoch 2/20


100%|██████████| 212/212 [00:30<00:00,  6.93it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.83
train_loss = 0.40
Epoch 3/20


...

100%|██████████| 212/212 [00:51<00:00,  4.09it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.94
train_loss = 0.15
Testing


100%|██████████| 23/23 [00:03<00:00,  5.78it/s]


test_acc = 0.97
test_loss = 0.10
================= Block 11 Finished ==============


: 

# 2. Training CNN modules

In [ ]:
from ReVIT.CNN import CNN

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = CNN(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/cnns/cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

# 3. Training ResNet_FT

In [ ]:
from ReVIT.ResnetFT import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/resnets_ft/resnet_ft_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

# 4. Training ResNet_TL

In [ ]:
from ReVIT.ResnetTL import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/resnets_tl/resnet_tl_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

# 5. Training ResNet_FT_CNN

In [ ]:
from ReVIT.ResnetFT_CNN import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/resnets_ft_cnn/resnet_ft_cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

# 6. Training ResNet_TL_CNN

In [ ]:
from ReVIT.ResnetTL_CNN import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "adv_data/TB_adversarial_data"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/adv_train_models/resnets_tl_cnn/resnet_tl_cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

In [ ]:
x = torch.randn(30, 196, 768)
# print(x.shape)

import torch.nn as nn
import torch

# This is what Faris told
class CNN(nn.Module):
    """
    CNN classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=128, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(in_features=3*3*64, out_features=128)
        self.relu3 = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        print(x.shape)
        # x = x.view(-1, 768, 14, 14) # reshape patches to image-like format
        x = x.reshape(-1, 768, 14, 14)
        # print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        print(x.shape)
        return x
    
import torch.nn as nn 
import torch

class Mlp(nn.Module): 
    """
    MLP classifier. 
    Args:
        num_classes -> number of classes 
        in_feature -> features dimension

    return logits. 
    
    """
    def __init__(self,num_classes=2 ,in_features = 768*196):
        
        super().__init__()
        self.linear1 = nn.Linear(in_features= in_features, out_features= 4096)
        self.linear2 = nn.Linear(in_features= 4096, out_features= 2048)
        self.linear3 = nn.Linear(in_features= 2048, out_features= 128)
        self.linear4 = nn.Linear(in_features= 128, out_features= num_classes)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self,x):
        print(x.shape)
        x= x.reshape(-1, 196*768)
        x = nn.functional.relu(self.linear1(x))
        x = nn.functional.relu(self.linear2(x))
        x = nn.functional.relu(self.linear3(x))
        x = self.linear4(x)
        print(x.shape)
        return x

import torch.nn as nn
import torchvision.models as models

class ResNet18_FT(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        print("HI")
        x = self.resnet18.layer1(x)
        print("HI")
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


class ResNet18_TL(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768, freeze=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)

        # Freeze layers
        if freeze:
            for param in self.resnet18.parameters():
                param.requires_grad = False

        self.resnet18.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


class ResNet18_FT_CNN(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        self.additional_convs = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.resnet18.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.additional_convs(x) # pass through additional conv layers
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x

import torch.nn as nn
import torchvision.models as models

class ResNet18_TL_CNN(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        
        # Freeze ResNet layers
        for param in self.resnet18.parameters():
            param.requires_grad = False
        
        self.additional_convs = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.resnet18.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.additional_convs(x) # pass through additional conv layers
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


# clf = CNN()
# # print(clf)
# output = clf(x)
# countParams(clf)

# mlp = Mlp()
# output = clf(x)
# countParams(mlp)

# resnet = ResNet18_FT()
# output = resnet(x)
# countParams(resnet)

# resnet = ResNet18_TL()
# output = resnet(x)
# countParams(resnet)

# resnet = ResNet18_FT_CNN()
# output = resnet(x)
# countParams(resnet)

resnet = ResNet18_TL_CNN()
output = resnet(x)
countParams(resnet)

# Testing

In [ ]:
def test_clfs(mlps_list, data_loader, mlp_root_dir, model):
    length = (len(mlps_list))
    print(length)
    # test_list = {f'block_{i}': [] for i in range(length)}
    # test_acc = 0.0
    # test_loss = 0.0
    # criterion = nn.CrossEntropyLoss()

    for index in range(length): 
        acc_avg = 0.0
        mlp_in = os.path.join(mlp_root_dir, mlps_list[index])
        print(mlp_in)
        print(mlp_in, f": MLP at index {index} has been loaded")
        classifier = torch.load(mlp_in).cuda()
        classifier.eval()
        data_loader_test = data_loader['test']

        with torch.no_grad(): 
            for image, label in tqdm(data_loader_test):
                image = image.cuda()
                label = label.cuda()
                x = model.patch_embed(image)
                x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
                x = model.pos_drop(x + model.pos_embed)
                for i in range(index+1):
                    x = model.blocks[i](x)
                output = classifier(x[:,1:,:])
                predictions = torch.argmax(output, dim=-1)
                acc = torch.sum(predictions == label).item()/len(label)
                acc_avg += acc
            print(f'Accuracy of block {index} = {(acc_avg/len(data_loader_test)):.3f}')
        print(f'================= Block {index} Finished ==============')


### \__main\__

In [ ]:
# __main__
# mlp
root_dir = "adv_data/TB_adversarial_data"
mlp_path = 'ReVIT/adv_train_models/mlps'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

mlps_list = sorted(os.listdir(mlp_path))
print(mlps_list)
acc = test_clfs(mlps_list= mlps_list, data_loader=loader_, mlp_root_dir=mlp_path, model=model)

In [ ]:
# __main__
# cnns
root_dir = "adv_data/TB_adversarial_data"
cnns_path = 'ReVIT/adv_train_models/cnns'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

cnns_list = sorted(os.listdir(cnns_path))
print(cnns_list)
acc = test_clfs(mlps_list= cnns_list, data_loader=loader_, mlp_root_dir=cnns_path, model=model)

In [ ]:
# __main__
# resnets_ft
root_dir = "adv_data/TB_adversarial_data"
resnets_ft_path = 'ReVIT/adv_train_models/resnets_ft'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

resnets_ft_list = sorted(os.listdir(resnets_ft_path))
print(resnets_ft_list)
acc = test_clfs(mlps_list= resnets_ft_list, data_loader=loader_, mlp_root_dir=resnets_ft_path, model=model)

In [ ]:
# __main__
# resnets_tl
root_dir = "adv_data/TB_adversarial_data"
resnets_tl_path = 'ReVIT/adv_train_models/resnets_tl'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

resnets_tl_list = sorted(os.listdir(resnets_tl_path))
print(resnets_tl_list)
acc = test_clfs(mlps_list= resnets_tl_list, data_loader=loader_, mlp_root_dir=resnets_tl_path, model=model)